In [78]:
import psycopg2
import os
import sys
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras

In [4]:
#import all necessary modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

# Import API key
import api_keys

# Get FBI API Key
API_KEY = api_keys.fbi_api_key

#Pull data from the website on trends for all crimes and all states from 2010 to 2019.

# base url for getting api data
base_url = "https://api.usa.gov/crime/fbi/sapi/"
#Create a list of states in order to request data from each state
stateAbbr = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
#Create a list of variables that we want to explore data for. In this case male and females
variable = 'all'
#declare the start and end year for the time period that we want to look at
since = '2010'
until = '2019'

#create an empty dataframe
crime_data = pd.DataFrame(columns=[])

#Create a for loop to request data for each state and for each variable for the offense that we would like to look at.
for i in stateAbbr:
    desired_endpoint = "api/arrest/states/offense/" + i + '/' + variable + '/' + since + '/' + until
    req = requests.get(base_url + desired_endpoint+'?api_key='+API_KEY).json()
    new = pd.DataFrame(req["data"])
    #Create new columns in the dataframe so that we know what data corresponds to each state and variable.
    new['State'] = i
    #Append the data from each request to the dataframe
    crime_data = crime_data.append(new)

In [7]:
import itertools
#base url
base_url = "https://api.usa.gov/crime/fbi/sapi/"
#Create a list of states in order to request data from each state
stateAbbr = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
offense = ["violent-crime", "property-crime"]
year = ['2010', "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", '2019']

#create an empty dataframe
summarized_crime_type_data = pd.DataFrame(columns = [])

#Create a for loop to request data for each state and for each variable for the offense that we would like to look at.
for i,j,k in itertools.product(stateAbbr, offense, year):
    desired_endpoint = "api/summarized/state/" + i + "/" + j + "/" + k + "/" + k
    req = requests.get(base_url + desired_endpoint + "?api_key=" + API_KEY).json()
    new = pd.DataFrame(req["results"])
    #Create new columns in the dataframe so that we know what data corresponds to each state and variable.
    #new["State"] = i
    #new["Offense type"] = j
    #new["Year"] = k
    #Append the data from each request to the dataframe
    summarized_crime_type_data = summarized_crime_type_data.append(new)

In [8]:
summarized_crime_type_data.head()

,ori,data_year,offense,state_abbr,cleared,actual
0,AL0010000,2010,violent-crime,AL,26,349
1,AL0010100,2010,violent-crime,AL,53,289
2,AL0010200,2010,violent-crime,AL,0,1343
3,AL0010400,2010,violent-crime,AL,21,105
4,AL0010500,2010,violent-crime,AL,12,30


In [99]:
summarized_crime_type_data.dtypes

ori           object
data_year      int64
offense       object
state_abbr    object
cleared        int64
actual         int64
dtype: object

In [97]:
summarized_crime_type_data.to_csv(r'Crime_type_data.csv')

In [100]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=CLS521")
cur = conn.cursor()
cur.execute('''CREATE TABLE Crime_Type
      (ori TEXT PRIMARY KEY,
      data_year INTEGER,
      offense TEXT,
      state_abbr TEXT,
      cleared INTEGER,
      actual INTEGER);''')
print("Table created successfully")

conn.commit()

Table created successfully


In [101]:
import csv

conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=CLS521")
cur = conn.cursor()
with open('crime_type_data.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        cur.execute(
        "INSERT INTO crime_data VALUES (%s, %s, %s, %s, %s, %s)",
        row
    )
conn.commit()

TypeError: not all arguments converted during string formatting